In [1]:
#Our original contribution will be in the form of a hard particle Monte Carlo
#simulation. The idea here is to use the HOOMD (Monte Carlo package) library
#to generate a rigid lattice with charged particles, and calculate
#the strain on the lattice when an external electric field is applied.
#Using this method we can draw some conclusions about the inverse piezo-
#electric effect where an electric field produced mechanical strain on the
#system.

#Team Catastrophe
#Joshua Milem & Anna Turnbull

from hoomd import *
import hoomd.md as md
import numpy as np
from pylab import *

hoomd.context.initialize("CPU");

#Creating our lattice. 16 x 16 2D sheet

unit_cell = hoomd.lattice.unitcell(N = 1, 
                                  a1 = [12,0,0],
                                  a2 = [0,1,0],
                                  a3 = [0,0,1],
                                  dimensions = 3,
                                  position = [[0,0,0]],
                                  type_name = ['R'],
                                  mass = [147.84558],
                                  moment_inertia = [[0,
                                                    1/12*1.0*8**2,
                                                    1/12*1.0*8**2]],
                                  orientation = [[1,0,0,0]]);
system = hoomd.init.create_lattice(unitcell=unit_cell, n=[1,16,16])

system.particles.types.add('A')
n1 = md.nlist.cell()
all = group.all()

#Create rigid bodies on either side of the 2D sheet to simulate crystal

crystallize = hoomd.md.constrain.rigid()
crystallize.set_param('R', types=['A']*4,
                     positions=[(-2,0,0),(-1,0,0),(1,0,0),(2,0,0)])


crystallize.create_bodies()

crystal = hoomd.group.rigid_center()

#NVT thermostat integration, may switch to Brownian to fix anisotropy error

md.integrate.mode_standard(dt=0.005)
md.integrate.nvt(crystal, kT=1.0, tau=0.05)

run(100)



%matplotlib inline
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

snapshot = system.take_snapshot(all=True)
pos = snapshot.particles.position

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
for part in pos:
    ax.scatter(part[0], part[1], part[2], c='b')
plt.show()



#Currently the issue is coupling the free particle layer to the rigid bodies.
#I do not think this will take much work, I just ran out of time for this 
#piece of the project. 

#HOOMD has an internal energy minimization function which will be used
#to calculate the final energy once the electric strain is applied,
#otherwise strain will be inaccurate.


RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

NameError: name 'hoomd' is not defined